In [ ]:
# Cài đặt thư viện Playwright
!pip install playwright

# Cài đặt các trình duyệt (Chromium, Firefox, Webkit)
!playwright install

# Cài đặt các thư viện hệ thống cần thiết cho Playwright
!playwright install-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 20.2 MB/s eta 0:00:00
(node:1171) [DEP0169] DeprecationWarning: `url.parse()` behavior is not standardized and prone to errors that have security implications. Use the WHATWG URL API instead. CVEs are not issued for `url.parse()` vulnerabilities.
(Use `node --trace-deprecation ...` to show where the warning was created)
167.3 MiB [] 0% 0.0s167.3 MiB [] 0% 172.0s167.3 MiB [] 0% 177.0s167.3 MiB [] 0% 539.5s167.3 MiB [] 0% 455.4s167.3 MiB [] 0% 612.8s167.3 MiB [] 0% 542.6s167.3 MiB [] 0% 474.8s167.3 MiB [] 0% 455.0s167.3 MiB [] 0% 423.4s167.3 MiB [] 0% 403.5s167.3 MiB [] 0% 388.2s167.3 MiB [] 0% 362.2s167.3 MiB [] 0% 355.5s167.3 MiB [] 0% 343.8s167.3 MiB [] 0% 320.2s167.3 MiB [] 0% 311.4s167.3 MiB [] 0% 292.0s167.3 MiB [] 0% 273.9s167.3 MiB [] 0% 248.5s167.3 MiB [] 0% 228.7s167.3 MiB [] 0% 212.9s167.3 MiB [] 0% 199.6s167.3 MiB [] 0% 188.3s167.3 MiB [] 0% 178.4s167.3 MiB [] 0% 170.4s167.3 MiB [] 0% 159.3s167.3 MiB [] 0% 149.9s167.3 M

In [ ]:
import sys
import random
import asyncio
import os
from playwright.async_api import async_playwright

async def run():
    async with async_playwright() as p:

        browser = await p.chromium.launch(
            headless=True,
            args=[
                "--no-sandbox",
                "--disable-dev-shm-usage"
            ]
        )

        context = await browser.new_context(
            user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
        )

        page = await context.new_page()

        base_url = "https://scholar.google.com.vn/scholar?hl=vi&q=khoa+học+máy+tính"
        max_articles = 10000
        data_list = []

        os.makedirs("pdf_files", exist_ok=True)

        for start in range(0, max_articles, 10):

            url = f"{base_url}&start={start}"
            print(f"\nĐang truy cập: {url}")

            try:
                await page.goto(url, timeout=30000)
                await page.wait_for_selector("div.gs_ri", timeout=10000)
            except:
                print("⚠ Có thể bị CAPTCHA")
                await asyncio.sleep(5)
                continue

            articles = page.locator("div.gs_r.gs_or.gs_scl")

            count = await articles.count()
            print(f"→ Tìm thấy {count} bài")

            for i in range(count):

                if len(data_list) >= max_articles:
                    break

                article = articles.nth(i)

                title = await article.locator("h3.gs_rt").inner_text()

                # ----------- LẤY LINK PDF BÊN PHẢI ------------
                pdf_link = None
                pdf_block = article.locator("div.gs_or_ggsm a")

                if await pdf_block.count() > 0:
                    pdf_link = await pdf_block.get_attribute("href")

                print(f"\n--- Bài {len(data_list)+1} ---")
                print("Tiêu đề:", title)
                print("PDF:", pdf_link)

                # ----------- TẢI PDF ------------
                if pdf_link:
                    try:
                        response = await context.request.get(pdf_link)
                        if response.status == 200:
                            file_path = f"pdf_files/paper_{len(data_list)+1}.pdf"
                            with open(file_path, "wb") as f:
                                f.write(await response.body())
                            print("✅ Đã tải:", file_path)
                        else:
                            print("❌ Không tải được (status != 200)")
                    except Exception as e:
                        print("❌ Lỗi tải PDF:", e)

                data_list.append(title)

                await asyncio.sleep(random.uniform(2, 4))

            await asyncio.sleep(random.uniform(4, 8))

        print(f"\n✅ ĐÃ LẤY TỔNG CỘNG: {len(data_list)} BÀI")
        await browser.close()


await run()


Đang truy cập: https://scholar.google.com.vn/scholar?hl=vi&q=khoa+học+máy+tính&start=0
→ Tìm thấy 10 bài

--- Bài 1 ---
Tiêu đề: Thiết kế chủ đề giáo dục STEM trong dạy học “Chuyên đề học tập Tin học 10-Định hướng khoa học máy tính”
PDF: https://tcgd.tapchigiaoduc.edu.vn/index.php/tapchi/article/download/1361/696
✅ Đã tải: pdf_files/paper_1.pdf

--- Bài 2 ---
Tiêu đề: KHẢO SÁT CÁC THUẬT TOÁN ĐỊNH HƯỚNG KHOA HỌC MÁY TÍNH MÔN TIN HỌC CHƯƠNG TRÌNH GIÁO DỤC PHỔ THÔNG MỚI
PDF: https://journal.hcmue.edu.vn/index.php/hcmuejos/article/download/2998/2823
✅ Đã tải: pdf_files/paper_2.pdf

--- Bài 3 ---
Tiêu đề: Đề xuất quy trình dạy học kết hợp trong dạy học mạch kiến thức “Khoa học máy tính” thuộc Chương trình giáo dục phổ thông môn Tin học 2018
PDF: https://tcgd.tapchigiaoduc.edu.vn/index.php/tapchi/article/download/762/513
✅ Đã tải: pdf_files/paper_3.pdf

--- Bài 4 ---
Tiêu đề: [PDF] Giải pháp tìm kiếm ngữ nghĩa cho văn bản khoa học máy tính tiếng Việt
PDF: https://www.academia.edu/download/9